# Lesson 6: Planning and Stock Report Generation

## Setup

In [2]:
# !pip install openai
# !pip install autogen

In [3]:
# reset all those keys
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
llm_config={"model": "gpt-4-turbo"}

## The task!

In [5]:
task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-04-23."

## Build a group chat

This group chat will include these agents:

1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

In [8]:
# !pip install dask

In [9]:
import autogen

In [10]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

In [11]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

In [12]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

**Note**: In this lesson, you'll use an alternative method of code execution by providing a dict config. However, you can always use the LocalCommandLineCodeExecutor if you prefer. For more details about code_execution_config, check this: https://microsoft.github.io/autogen/docs/reference/agentchat/conversable_agent/#__init__

In [13]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [14]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

## Define the group chat

In [15]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [16]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


## Start the group chat!

<p style="background-color:#ECECEC; padding:15px; "> <b>Note:</b> In this lesson, you will use GPT 4 for better results. Please note that the lesson has a quota limit. If you want to explore the code in this lesson further, we recommend trying it locally with your own API key.

In [22]:
# !pip install beautifulsoup4
# !pip install lxml
# !pip install yfinance
# # !pip install newsapi-python # will be asked to restart session

In [109]:
import yfinance as yf

def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data


nvidia_data = fetch_stock_data('NVDA', '2024-03-23', '2024-04-24')
print(nvidia_data)

[*********************100%***********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2024-03-25  93.941002  96.765999  93.510002  95.001999  94.986069  552136000
2024-03-26  95.850998  96.375000  92.501999  92.560997  92.545479  513648000
2024-03-27  93.112000  93.239998  89.123001  90.250000  90.234863  586067000
2024-03-28  90.000000  91.300003  89.193001  90.356003  90.340851  435212000
2024-04-01  90.299004  92.224998  89.204002  90.362999  90.347847  452441000
2024-04-02  88.447998  90.094002  87.620003  89.452003  89.437004  433064000
2024-04-03  88.484001  90.374001  88.400002  88.963997  88.949081  370067000
2024-04-04  90.405998  90.634003  85.879997  85.904999  85.890594  434965000
2024-04-05  86.865997  88.481003  85.926003  88.008003  87.993248  399678000
2024-04-08  88.699997  88.830002  86.732002  87.133003  87.118393  283220000
2024-04-09  87.442001  87.635002  83.022003  85.353996  85.339684  503547000

In [24]:
# !pip install newsapi-python

In [110]:
from newsapi import NewsApiClient

def fetch_news_articles(api_key, query, from_date, to_date):
    newsapi = NewsApiClient(api_key=api_key)
    articles = newsapi.get_everything(q=query, from_param=from_date, to=to_date, language='en')
    return articles

api_key = os.getenv("NEWS_API_KEY")
nvidia_news = fetch_news_articles(api_key, 'Nvidia', '2024-10-15', '2024-09-15') # there is actually a date range limit on free account
print(nvidia_news)

{'status': 'ok', 'totalResults': 6921, 'articles': [{'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Jeremy Gan', 'title': 'The NVIDIA app, currently in beta, will replace GeForce Experience soon', 'description': 'This February, NVIDIA released a one-stop graphical adjustments app in beta, succinctly named the NVIDIA App\r\n. The app isn’t a direct replacement for the existing GeForce Experience app since it also lets you install other NVIDIA apps like GeForce NOW (cloud…', 'url': 'https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_9cb0a864-a4f9-4adb-90fa-941b842c7d1a', 'urlToImage': None, 'publishedAt': '2024-10-02T15:28:53Z', 'content': "If you click 'Accept all', we and our partners, including 240 who are part of the IAB Transparency &amp; Consent Framework, will also store and/or access information on a device (in other words, use … [+678 chars]"}, {'source': {'id': None, 'name': 'Gizmodo.com'}, 'author': 'Kyle Barr', 'title': 'The Leaked Nvidia RT

In [28]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia in the past month, you need the following information:

1. Historical stock prices of Nvidia for the past month (from 2024-03-23 to 2024-04-23).
2. Significant events or news related to Nvidia during this period that could have impacted stock prices.
3. General market trends during this period that could have influenced Nvidia's stock performance.

### Steps to Acquire Data Using Python:

#### Step 1: Get Historical Stock Prices
- **Tools Required:** Use libraries like `yfinance` to fetch historical stock prices.
- **Data Points Needed:** Daily open, high, low, close, and volume for Nvidia.

```python
import yfinance as yf

def f

In [113]:
# market_index_data = fetch_stock_data('NVDA', '2024-09-15', '2024-10-15')
# print(market_index_data)

In [88]:
# filename: fetch_nvidia_news.py
from newsapi import NewsApiClient

def fetch_news_articles(api_key, query, from_date, to_date):
    newsapi = NewsApiClient(api_key=api_key)
    articles = newsapi.get_everything(q=query, from_param=from_date, to=to_date, language='en')
    return articles

api_key = os.getenv("NEWS_API_KEY")
nvidia_news = fetch_news_articles(api_key, 'Nvidia', '2024-10-14', '2024-09-15')
print(nvidia_news)

{'status': 'ok', 'totalResults': 6627, 'articles': [{'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Jeremy Gan', 'title': 'The NVIDIA app, currently in beta, will replace GeForce Experience soon', 'description': 'This February, NVIDIA released a one-stop graphical adjustments app in beta, succinctly named the NVIDIA App\r\n. The app isn’t a direct replacement for the existing GeForce Experience app since it also lets you install other NVIDIA apps like GeForce NOW (cloud…', 'url': 'https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_9cb0a864-a4f9-4adb-90fa-941b842c7d1a', 'urlToImage': None, 'publishedAt': '2024-10-02T15:28:53Z', 'content': "If you click 'Accept all', we and our partners, including 240 who are part of the IAB Transparency &amp; Consent Framework, will also store and/or access information on a device (in other words, use … [+678 chars]"}, {'source': {'id': None, 'name': 'Gizmodo.com'}, 'author': 'Kyle Barr', 'title': 'The Leaked Nvidia RT

In [106]:
import yfinance as yf

def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

snap_data = fetch_stock_data('^GSPC', '2024-09-15', '2024-10-14')
print(snap_data)

[*********************100%***********************]  1 of 1 completed

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2024-09-16  5615.209961  5636.049805  5604.529785  5633.089844  5633.089844   
2024-09-17  5655.509766  5670.810059  5614.049805  5634.580078  5634.580078   
2024-09-18  5641.680176  5689.750000  5615.080078  5618.259766  5618.259766   
2024-09-19  5702.629883  5733.569824  5686.419922  5713.640137  5713.640137   
2024-09-20  5709.640137  5715.140137  5674.490234  5702.549805  5702.549805   
2024-09-23  5711.899902  5725.359863  5704.220215  5718.569824  5718.569824   
2024-09-24  5727.660156  5735.319824  5698.990234  5732.930176  5732.930176   
2024-09-25  5733.649902  5741.029785  5712.060059  5722.259766  5722.259766   
2024-09-26  5762.220215  5767.370117  5721.009766  5745.370117  5745.370117   
2024-09-27  5755.359863  5763.779785  5727.339844  5738.169922  5738.169922   
2024-09-30  5726.520020  5765.140137  5703.529785  5

In [99]:
from newsapi import NewsApiClient

def fetch_news_articles(api_key, query, from_date, to_date):
    newsapi = NewsApiClient(api_key=api_key)
    articles = newsapi.get_everything(q=query, from_param=from_date, to=to_date, language='en')
    return articles

apī_key = os.getenv("NEWS_API_KEY")
snap_news = fetch_news_articles(api_key, 'GSPC', '2024-10-14', '2024-09-15')
print(snap_news)

{'status': 'ok', 'totalResults': 102, 'articles': [{'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Josh Lipton', 'title': 'Q3 earnings focus is on price pressures: JPMorgan strategist', 'description': 'After the S&P 500 (^GSPC) and Dow Jones Industrial Average (^DJI) close at record highs, JPMorgan asset management global market strategist Jordan Jackson...', 'url': 'https://finance.yahoo.com/video/q3-earnings-focus-price-pressures-223026468.html', 'urlToImage': 'https://s.yimg.com/cv/apiv2/social/images/yahoo_default_logo.png', 'publishedAt': '2024-10-14T22:30:26Z', 'content': 'After the S&amp;P 500 (^GSPC) and Dow Jones Industrial Average (^DJI) close at record highs, JPMorgan asset management global market strategist Jordan Jackson sits down with Market Domination Overtim… [+2539 chars]'}, {'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Yahoo Finance', 'title': "Stocks close lower, what's next for the Fed: Market Domination Overtime", 'descripti

## Add a speaker selection policy

In [100]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan "
    "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [102]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [103]:
# import yfinance as yf

# def fetch_stock_data(ticker, start_date, end_date):
#     stock_data = yf.download(ticker, start=start_date, end=end_date)
#     return stock_data

# nvidia_data = fetch_stock_data('NVDA', '2024-09-15', '2024-10-15')

[*********************100%***********************]  1 of 1 completed


In [104]:
# from newsapi import NewsApiClient

# def fetch_news_articles(api_key, query, from_date, to_date):
#     newsapi = NewsApiClient(api_key=api_key)
#     articles = newsapi.get_everything(q=query, from_param=from_date, to=to_date, language='en')
#     return articles

# # apī_key = os.getenv("NEWS_API_KEY")
# nvidia_news = fetch_news_articles(api_key, 'Nvidia', '2024-10-15', '2024-09-15')

In [105]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia in the past month, you need the following information:

1. Historical stock prices of Nvidia for the past month (from 2024-03-23 to 2024-04-23).
2. Significant events or news related to Nvidia during this period that could have impacted stock prices.
3. General market trends during this period that could have influenced Nvidia's stock performance.

### Steps to Acquire Data Using Python:

#### Step 1: Get Historical Stock Prices
- **Tools Required:** Use libraries like `yfinance` to fetch historical stock prices.
- **Data Points Needed:** Daily open, high, low, close, and volume for Nvidia.

```python
import yfinance as yf

def f

In [114]:
import yfinance as yf

def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

nvidia_data = fetch_stock_data('NVDA', '2024-09-15', '2024-10-15')
print(nvidia_data)

[*********************100%***********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-09-16  116.790001  118.180000  114.360001  116.779999  116.779999   
2024-09-17  118.169998  118.800003  114.830002  115.589996  115.589996   
2024-09-18  115.889999  117.699997  113.220001  113.370003  113.370003   
2024-09-19  117.349998  119.660004  117.250000  117.870003  117.870003   
2024-09-20  117.059998  118.620003  115.389999  116.000000  116.000000   
2024-09-23  116.550003  116.989998  114.860001  116.260002  116.260002   
2024-09-24  116.519997  121.800003  115.379997  120.870003  120.870003   
2024-09-25  122.019997  124.940002  121.610001  123.510002  123.510002   
2024-09-26  126.800003  127.669998  121.800003  124.040001  124.040001   
2024-09-27  123.970001  124.029999  119.260002  121.400002  121.400002   
2024-09-30  118.309998  121.500000  118.150002  121.440002  121.440002   
2024-10-01  121.769997  122.440002  11

In [115]:
from newsapi import NewsApiClient

def fetch_news_articles(api_key, query, from_date, to_date):
    newsapi = NewsApiClient(api_key=api_key)
    articles = newsapi.get_everything(q=query, from_param=from_date, to=to_date, language='en')
    return articles

apī_key = os.getenv("NEWS_API_KEY")
nvidia_news = fetch_news_articles(api_key, 'Nvidia', '2024-10-15', '2024-09-15')
print(nvidia_news)

{'status': 'ok', 'totalResults': 6921, 'articles': [{'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Jeremy Gan', 'title': 'The NVIDIA app, currently in beta, will replace GeForce Experience soon', 'description': 'This February, NVIDIA released a one-stop graphical adjustments app in beta, succinctly named the NVIDIA App\r\n. The app isn’t a direct replacement for the existing GeForce Experience app since it also lets you install other NVIDIA apps like GeForce NOW (cloud…', 'url': 'https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_9cb0a864-a4f9-4adb-90fa-941b842c7d1a', 'urlToImage': None, 'publishedAt': '2024-10-02T15:28:53Z', 'content': "If you click 'Accept all', we and our partners, including 240 who are part of the IAB Transparency &amp; Consent Framework, will also store and/or access information on a device (in other words, use … [+678 chars]"}, {'source': {'id': None, 'name': 'Gizmodo.com'}, 'author': 'Kyle Barr', 'title': 'The Leaked Nvidia RT

In [116]:
market_index_data = fetch_stock_data('^GSPC', '2024-09-15', '2024-10-15',)

[*********************100%***********************]  1 of 1 completed


In [118]:
# filename: fetch_nvidia_stock_data.py
import yfinance as yf

def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

# Fetching Nvidia's stock data
snap_data = fetch_stock_data('^GSPC', '2024-03-23', '2024-04-24')
print(snap_data)

[*********************100%***********************]  1 of 1 completed

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2024-03-25  5219.520020  5229.089844  5216.089844  5218.189941  5218.189941   
2024-03-26  5228.850098  5235.160156  5203.419922  5203.580078  5203.580078   
2024-03-27  5226.310059  5249.259766  5213.919922  5248.490234  5248.490234   
2024-03-28  5248.029785  5264.850098  5245.819824  5254.350098  5254.350098   
2024-04-01  5257.970215  5263.950195  5229.200195  5243.770020  5243.770020   
2024-04-02  5204.290039  5208.339844  5184.049805  5205.810059  5205.810059   
2024-04-03  5194.370117  5228.750000  5194.370117  5211.490234  5211.490234   
2024-04-04  5244.049805  5256.589844  5146.060059  5147.209961  5147.209961   
2024-04-05  5158.950195  5222.180176  5157.209961  5204.339844  5204.339844   
2024-04-08  5211.370117  5219.569824  5197.350098  5202.390137  5202.390137   
2024-04-09  5217.029785  5224.810059  5160.779785  5

In [121]:
# filename: fetch_nvidia_news.py
from newsapi import NewsApiClient

def fetch_news_articles(api_key, query, from_date, to_date):
    newsapi = NewsApiClient(api_key=api_key)
    articles = newsapi.get_everything(q=query, from_param=from_date, to=to_date, language='en')
    return articles['articles']  # Assuming you only need the articles part

apī_key = os.getenv("NEWS_API_KEY")
snap_data = fetch_news_articles(api_key, 'GSPC', '2024-10-15', '2024-09-15')
print(snap_data)

[{'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Josh Lipton', 'title': 'Q3 earnings focus is on price pressures: JPMorgan strategist', 'description': 'After the S&P 500 (^GSPC) and Dow Jones Industrial Average (^DJI) close at record highs, JPMorgan asset management global market strategist Jordan Jackson...', 'url': 'https://finance.yahoo.com/video/q3-earnings-focus-price-pressures-223026468.html', 'urlToImage': 'https://s.yimg.com/cv/apiv2/social/images/yahoo_default_logo.png', 'publishedAt': '2024-10-14T22:30:26Z', 'content': 'After the S&amp;P 500 (^GSPC) and Dow Jones Industrial Average (^DJI) close at record highs, JPMorgan asset management global market strategist Jordan Jackson sits down with Market Domination Overtim… [+2539 chars]'}, {'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Yahoo Finance', 'title': "Stocks close lower, what's next for the Fed: Market Domination Overtime", 'description': 'On the latest Market Domination Overtime: Th

**Note**: You might experience slightly different interactions between the agents. The engineer agent may write incorrect code, which the executor agent will report and send back for correction. This process could go through multiple rounds.